In [2]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optimizers
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

# GANモデルのクラス
class GAN(nn.Module):
    def __init__(self, device='cpu'):
        super().__init__()
        self.device = device
        self.G = Generator(device=device)
        self.D = Discriminator(device=device)
    def forward(self, x):
        x = self.G(x)
        y = self.D(x)
        return y
    
# 識別器(Generator)のクラス
class Discriminator(nn.Module):
    def __init__(self, device='cpu'):
        super().__init__()
        self.device = device
        self.conv1 = nn.Conv2d(1, 128,
                               kernel_size=(3, 3),
                               stride=(2, 2),
                               padding=1)
        self.relu1 = nn.LeakyReLU(0.2)
        self.conv2 = nn.Conv2d(128, 256,
                               kernel_size=(3, 3),
                               stride=(2, 2),
                               padding=1)
        self.bn2 = nn.BatchNorm2d(256)
        self.relu2 = nn.LeakyReLU(0.2)
        self.fc = nn.Linear(256*7*7, 1024)
        self.bn3 = nn.BatchNorm1d(1024)
        self.relu3 = nn.LeakyReLU(0.2)
        self.out = nn.Linear(1024, 1)

    def forward(self, x):
        h = self.conv1(x)
        h = self.relu1(h)
        h = self.conv2(h)
        h = self.bn2(h)
        h = self.relu2(h)
        h = h.view(-1, 256*7*7)
        h = self.fc(h)
        h = self.bn3(h)
        h = self.relu3(h)
        h = self.out(h)
        y = torch.sigmoid(h)
        return y

# 生成器(Discriminator)のクラス
class Generator(nn.Module):
    def __init__(self, input_dim=100, device='cpu'):
        super().__init__()
        self.device = device
        self.linear = nn.Linear(input_dim, 256*14*14)
        self.bn1 = nn.BatchNorm1d(256*14*14)
        self.relu1 = nn.ReLU()
        self.conv1 = nn.Conv2d(256, 128,
                               kernel_size=(3, 3),
                               padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.relu2 = nn.ReLU()
        self.conv2 = nn.Conv2d(128, 64,
                               kernel_size=(3, 3),
                               padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.relu3 = nn.ReLU()
        self.conv3 = nn.Conv2d(64, 1, kernel_size=(1, 1))
    
    def forward(self, x):
        h = self.linear(x)
        h = self.bn1(h)
        h = self.relu1(h)
        h = h.view(-1, 256, 14, 14)
        h = F.interpolate(h, size=(28, 28))
        h = self.conv1(h)
        h = self.bn2(h)
        h = self.relu2(h)
        h = self.conv2(h)
        h = self.bn3(h)
        h = self.relu3(h)
        h = self.conv3(h)
        y = torch.sigmoid(h)
        return y
    
#一様乱数のノイズを生成する関数
def gen_noise(batch_size):
    return torch.empty(batch_size, 100).uniform_(0, 1).to(device)
    
if __name__ == '__main__':
    # ランダムシードの固定
    np.random.seed(1234)
    torch.manual_seed(1234)
    # 演算デバイスの設定、GPUが使用可能ならGPUを使用、そうでなければCPUを使用
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    #### 1. データの読み込み
    # Fashion-MNIST: MNISTのファッションデータ
    # 60,000 例のトレーニング セットと 10,000 例のテスト セット
    # 28x28 グレースケール イメージで、10 クラスのラベル
    root = os.path.join(os.path.dirname(__file__), '.', 'data', 'fashion_mnist')
    # 複数の前処理を定義, 画像データをテンソルに変換する, 1次元のベクトルに変換するラムダ関数
    transform = transforms.Compose([transforms.ToTensor(),lambda x: x.view(-1)])
    # Fashion-MNISTのデータを前処理して、ダウンロードする
    mnist_train = torchvision.datasets.FashionMNIST(root=root,
                                                    download=True,
                                                    train=True,
                                                    transform=transform)
    # 訓練データをバッチで取得するためのデータローダ
    # 1つのバッチに含まれるサンプル数が100
    train_dataloader = DataLoader(mnist_train,batch_size=100,shuffle=True)

    #### 2. モデルの構築
    # モデルの定義
    model = GAN(device=device).to(device)

    # 最適化手法の定義
    # Adam(Adaptive moment): 適応的モーメント, モーメンタムとRMSPropを組み合わせて、鞍点に落ち込みにくく、学習率を0にしにくくしている
    # Momentum: 運動量, 勾配に落ち込む際に加速度項を追加して、物理的な加速度を再現して、鞍点に落ち込むのを防ぐ作用
    # AdaGrad(Adaptive Gradient): 適応的勾配, 重みwの学習率の自動調整, k方向の累積の修正量が多い場合はk方向の修正量を減らす作用
    # RMS Prop: Root Mean Square Prop, 重みの移動平均, 過去の情報が指数関数的に薄まって作用
    optimizer_D = optimizers.Adam(model.D.parameters(), lr=0.0002) # learning_rate: 学習率
    optimizer_G = optimizers.Adam(model.G.parameters(), lr=0.0002)

    # 損失関数の定義
    # 交差エントロピー誤差　Binary Cross Entropy Loss {H(p, q) = -\sum_{x} p(x) log(q(x))} 微分計算がしやすい
    criterion = nn.BCELoss()
    def compute_loss(label, preds):
        return criterion(preds, label)

    #### 3. モデルの訓練   
    def train_step(x):
        # 入力データのバッチサイズ
        batch_size = x.size(0)
        # 識別器と生成器の訓練モードを有効にする
        model.D.train()
        model.G.train()
        # 識別器の訓練
        # 784 次元のベクトルを 28x28 の画像に変更
        # x.size() = 100 × 784の行列
        x = x.view(-1, 1, 28, 28)
        # x.size() = 28 × 28の行列, チャンネル数1(グレースケール)
        
        # 入力データに対する推論をして、次元を削減して1次元のテンソルに変換
        preds = model.D(x).squeeze() # 本物画像に対する予測
        # 本物の画像に対する正解ラベルとして、サイズが batch_size の値1のテンソル
        t = torch.ones(batch_size).float().to(device)
        # 予測値 preds と本物正解ラベル t を用いて交差エントロピー誤差を計算
        loss_D_real = compute_loss(t, preds)

        # 偽物画像生成
        noise = gen_noise(batch_size)
        # 生成器モデルにノイズ noise を入力し、生成された偽物の画像
        gen = model.G(noise)
        # 生成器モデルにノイズ noise を入力し、生成された偽物の画像に対する推論, 生成器Gに勾配が伝わらないようにするため,detachしている。
        preds = model.D(gen.detach()).squeeze()
        # 偽物の画像に対する正解ラベルとして、サイズが batch_size の0のテンソル
        t = torch.zeros(batch_size).float().to(device)
        # 予測値 preds と偽物正解ラベル t を用いて交差エントロピー誤差を計算
        loss_D_fake = compute_loss(t, preds)
        
        # 識別器と生成器の損失関数の合算
        loss_D = loss_D_real + loss_D_fake
        
        # モデル内のパラメータの勾配を初期化して、識別機の学習
        optimizer_D.zero_grad()
        loss_D.backward()
        optimizer_D.step()

        # 生成器の学習
        noise = gen_noise(batch_size)
        gen = model.G(noise)
        preds = model.D(gen).squeeze()
        # 生成器の出力はすべて正解として損失関数を計算
        t = torch.ones(batch_size).float().to(device)
        loss_G = compute_loss(t, preds)
        # モデル内のパラメータの勾配を初期化して、生成器の学習
        optimizer_G.zero_grad()
        loss_G.backward()
        optimizer_G.step()

        return loss_D, loss_G

    epochs = 10
    for epoch in range(epochs):
        train_loss_D = 0.
        train_loss_G = 0.
        test_loss = 0.
        for (x, _) in train_dataloader:
            x = x.to(device)
            loss_D, loss_G = train_step(x)
            train_loss_D += loss_D.item()
            train_loss_G += loss_G.item()
        train_loss_D /= len(train_dataloader)
        train_loss_G /= len(train_dataloader)
        print('Epoch: {}, D Cost: {:.3f}, G Cost: {:.3f}'.format(epoch+1,train_loss_D,train_loss_G))

    # モデルのパラメータのみを保存
    torch.save(model.state_dict(), 'model_weight.pth')
    # モデル全体を保存
    torch.save(model, 'model.pth')

    ### 4. Test model
    # 保存したモデルの読み込み
    model = GAN(device=device).to(device)
    model.load_state_dict(torch.load('model_weight.pth'))
    # 特定のバッチサイズで一様乱数の画像を生成して、生成器で推論する関数
    def generate(batch_size=16):
        model.eval()
        noise = gen_noise(batch_size)
        gen = model.G(noise)
        return gen
    images = generate(batch_size=16)
    # 次元を削減して1次元のテンソルに変換して、GPUからCPUにデータを移動する
    images = images.squeeze().detach().cpu().numpy()
    # 生成した画像を一つのキャンバスに並べて表示する
    plt.figure(figsize=(6, 6))
    for i, image in enumerate(images):
        plt.subplot(4, 4, i+1)
        plt.imshow(image, cmap='binary_r')
        plt.axis('off')
        plt.tight_layout()
    plt.savefig("GAN-Fashion-MNIST.jpg")
    plt.show()


NameError: name '__file__' is not defined